In [1]:
from nltk import bigrams,trigrams, download 
from nltk.corpus import reuters
from gensim.test.utils import datapath
from gensim.corpora import WikiCorpus

from collections import Counter, defaultdict

In [2]:
path_for_wiki_dataset = datapath('enwiki-latest-pages-articles1.xml-p000000010p000030302-shortened.bz2')
wiki_sentences = WikiCorpus(path_for_wiki_dataset).get_texts()

download('punkt')
download('reuters')
reuters_sentences  = reuters.sents()

print(wiki_sentences)
print(reuters_sentences)

c:\users\tonoy\appdata\local\pypoetry\cache\virtualenvs\ai-bmuzw5yg-py3.7\lib\site-packages\gensim\utils.py:1268: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected %s; aliasing chunkize to chunkize_serial" % entity)
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tonoy\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\tonoy\AppData\Roaming\nltk_data...
[nltk_data]   Package reuters is already up-to-date!


<generator object WikiCorpus.get_texts at 0x00000234ADCDBEC8>
[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [3]:
def calculate_probabilities(sentence_model):
  for next_word in sentence_model:
    next_words = sentence_model[next_word]
    total_word_count = float(sum(next_words.values()))
    for previous_word in next_words:
      next_words[previous_word] /= total_word_count
    
def calculate_sigle_word_probability(sentence_model, word_count):
  for word in sentence_model:
    sentence_model[word] /= word_count

def convert_to_lower(word):
  if isinstance(word, str):
    return word.lower()
  return word

In [4]:
sentence_model_1 = defaultdict(lambda:0)
sentence_model_2 = defaultdict(lambda: defaultdict(lambda:0))
sentence_model_3 = defaultdict(lambda: defaultdict(lambda:0))
sentence_model_4 = defaultdict(lambda: set())
sentence_model_5 = defaultdict(lambda: set())

def calculate_word_count(sentence_model_1, sentence_model_2, sentence_model_3, sentences):
  word_count = 0
  for sentence in sentences:
    for word in sentence:
      word_count += 1
      sentence_model_1[word] += 1
    for previous_word_2, previous_word_1, next_word in trigrams(sentence, pad_right=True, pad_left=True):
      previous_word_1 = convert_to_lower(previous_word_1)
      previous_word_2 = convert_to_lower(previous_word_2)
        
      next_word = convert_to_lower(next_word)
      sentence_model_2[next_word][previous_word_1] += 1
      sentence_model_3[next_word][previous_word_2] += 1
      sentence_model_4[previous_word_1].add(next_word)
      sentence_model_5[previous_word_2].add(next_word)
      
  return word_count

In [5]:
wiki_word_count = calculate_word_count(sentence_model_1,sentence_model_2,sentence_model_3,wiki_sentences)
print(wiki_word_count)

452944


In [6]:
reuters_word_count = calculate_word_count(sentence_model_1,sentence_model_2,sentence_model_3,reuters_sentences)
print(reuters_sentences)

[['ASIAN', 'EXPORTERS', 'FEAR', 'DAMAGE', 'FROM', 'U', '.', 'S', '.-', 'JAPAN', 'RIFT', 'Mounting', 'trade', 'friction', 'between', 'the', 'U', '.', 'S', '.', 'And', 'Japan', 'has', 'raised', 'fears', 'among', 'many', 'of', 'Asia', "'", 's', 'exporting', 'nations', 'that', 'the', 'row', 'could', 'inflict', 'far', '-', 'reaching', 'economic', 'damage', ',', 'businessmen', 'and', 'officials', 'said', '.'], ['They', 'told', 'Reuter', 'correspondents', 'in', 'Asian', 'capitals', 'a', 'U', '.', 'S', '.', 'Move', 'against', 'Japan', 'might', 'boost', 'protectionist', 'sentiment', 'in', 'the', 'U', '.', 'S', '.', 'And', 'lead', 'to', 'curbs', 'on', 'American', 'imports', 'of', 'their', 'products', '.'], ...]


In [7]:
total_word = wiki_word_count + reuters_word_count
calculate_sigle_word_probability(sentence_model_1,total_word)

In [8]:
max_probability_words = []
def make_word_suggestion_by_trigram(previous_word_2, previous_word_1):
  for next_word in sentence_model_4[previous_word_1] and sentence_model_5[previous_word_2]:
    naive_bias_trigram_weight = sentence_model_1[next_word] * sentence_model_2[next_word][previous_word_1] * sentence_model_3[next_word][previous_word_2]
    max_probability_words.append((next_word,naive_bias_trigram_weight))

In [9]:
make_word_suggestion_by_trigram('harry','potter')
max_probability_words.sort(key=lambda o:o[1],reverse=True)
print(*max_probability_words[:15])

('and', 0.3644483248928979) ('seventh', 0.0) ('meehan', 0.0) ('street', 0.0) ('frank', 0.0) ('by', 0.0) ('empire', 0.0) ('fosdick', 0.0) ('released', 0.0) (',', 0.0) ('russell', 0.0) ('mon', 0.0) ('victor', 0.0) ('for', 0.0) ('told', 0.0)


In [ ]:
def suggest_word(word_list):
    make_word_suggestion_by_trigram(text[0].lower(), text[1].lower())
    max_probability_words.sort(key=lambda o:o[1], reverse=True)
    print(*max_probability_words[:15])

In [ ]:
while(True):
    text = input("Enter comma separated word(max 2): ")
    if text == "stop":
        print("....Terminated....")
        break
    else:
        try:
            text = text.split(",")
            suggest_word(text)
        except:
            continue

Enter comma separated word(max 2):  linux,windows


('the', 1.8677716744538864) ('of', 1.0056208745637372) ('it', 0.6233747235908828) (',', 0.3661503656397534) ('and', 0.3644483248928979) ('to', 0.3445965496413984) ('to', 0.3445965496413984) ('and', 0.16400174620180408) ('in', 0.1613934837600012) ('for', 0.15843745299262466) ('said', 0.0699400743653803) ('is', 0.04701680558232564) ('with', 0.04191160336378453) ('and', 0.03644483248928979) ('as', 0.011532936098490197) ('he', 0.004660831580308032) ('has', 0.0027172850518041402) ('down', 0.000443450616207752) ('nothing', 5.0141200380337106e-05) ('assume', 2.0700495569863942e-05)


Enter comma separated word(max 2):  harry,potter


('the', 1.8677716744538864) ('of', 1.0056208745637372) ('it', 0.6233747235908828) (',', 0.3661503656397534) ('and', 0.3644483248928979) ('and', 0.3644483248928979) ('to', 0.3445965496413984) ('to', 0.3445965496413984) ('and', 0.16400174620180408) ('in', 0.1613934837600012) ('for', 0.15843745299262466) ('said', 0.0699400743653803) ('is', 0.04701680558232564) ('with', 0.04191160336378453) ('and', 0.03644483248928979) ('as', 0.011532936098490197) ('he', 0.004660831580308032) ('has', 0.0027172850518041402) ('down', 0.000443450616207752) ('nothing', 5.0141200380337106e-05)
